In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

In [2]:
d = {'a': 1, 'b':2}
[print(d[key]) for key in d]

1
2


[None, None]

# Creating a dictionary

In [28]:
from os import walk, path
dic_csv = {}
df_var = {} 
for root,d_names,f_names in walk("./data/"):
    for f in f_names:
        filename = f.rsplit('.', 1)[0]
        dic_csv[path.join(root, f)] = ('processed_' + filename) if ('processed' in root) else filename

    

In [29]:
def make_df(dic):
    for key in dic:
        df_var[dic[key]] = pd.read_csv(key, delimiter=";")
make_df(dic_csv)

In [34]:
print(df_var['client'])

      client_id  birth_number  district_id
0             1        706213           18
1             2        450204            1
2             3        406009            1
3             4        561201            5
4             5        605703            5
...         ...           ...          ...
5364      13955        456030            1
5365      13956        430406            1
5366      13968        680413           61
5367      13971        626019           67
5368      13998        535812           74

[5369 rows x 3 columns]


In [ ]:
def separate(df, column):
    list = df[column].unique().tolist()
    dic = {}
    for l in list:
        dic[l] = df[df[column]==l]
    return dic

In [ ]:
def concat_train_test(dic):
    lis = [d for d in list(dic.values()) if ('test' in d or 'train' in d)]
    train = []
    test = []
    names = []
    for l in lis:
        names.append(l.split("_")[0])
        if (l.split("_")[1] == 'train'):
            train.append(l)
        else:
            test.append(l)
    names = list(set(names))
    
    for n in names:
        t1 = [d for d in train if n in d][0]
        t2 = [d for d in test if n in d][0]
        
        df_var[n] = pd.concat([df_var[t1],df_var[t2]])
concat_train_test(dic_csv)

# Auxiliary functions to process data

In [ ]:
from datetime import date

def getDate(d):
    year = 1900+int(str(d)[0:2])
    month = int(str(d)[2:4])
    gender = 'M' if (month < 50) else 'F'
    month = month if (month < 50) else month - 50
    day = int(str(d)[4:6])
    return {'year': year, 'month': month, 'day': day, 'gender': gender}

def getGender(df):
    list = []
    for row in df.itertuples(index = True):
        d = getDate(row.birth_number)
        birth_number = row.birth_number 
        if (d['gender'] == 'F'):
            birth_number -= 5000
        list.append([row.client_id, birth_number, d['gender'], row.district_id])
    
    return pd.DataFrame(list, columns=['client_id', 'birth_number', 'gender', 'district_id'])

print(getGender(df_var['client']))


In [ ]:
def date_range(df,min_year, max_year):
    
    

In [ ]:
def getColumns(df):
    sum = 0
    for key in df:
        if not 'train' in key and not 'test' in key:
            print('{} -> {}'.format(key, len(df_var[key].columns)))
            print('{}\n'.format(df_var[key].columns))

            sum = sum + len(df_var[key].columns)
    print(sum)

In [ ]:
#credit in cash', 'collection from another bank', nan,'withdrawal in cash', 'remittance to another bank',
#       'credit card withdrawal'

def n_credit_cash(col):
    return sum(col == 'credit in cash')

def n_collection_from_another_bank(col):
    return sum(col == 'collection from another bank')

def n_withdrawal_in_cash(col):
    return sum(col == 'withdrawal in cash')

def n_remittance_to_another_bank(col):
    return sum(col == 'remittance to another bank')

def n_credit_card_withdrawal(col):
    return sum(col == 'credit card withdrawal')

def n_interest_credited(col):
    return sum(col == 'none')



def n_credit(col):
    return sum(col == 'credit')

def n_withdrawal(col):
    return sum(col == 'withdrawal') + sum(col == 'withdrawal in cash')

# Merging information

In [ ]:
def Merge():
    # Copying information from dictonary into dataframes

    client = df_var['client'].copy()
    disp = df_var['disp'].copy()
    card = df_var['card'].copy()
    loan = df_var['loan'].copy()
    account = df_var['account'].copy()
    district = df_var['district'].copy()
    card = card.rename({'type': 'card_type'}, axis=1)
    disp = disp.rename({'type': 'disp_type'}, axis=1)

    # Merging informations

    merge1 = pd.merge(client, disp[disp['disp_type'] =='OWNER'], on ='client_id')
    merge2 = pd.merge(merge1, account, on = 'account_id')
    merge3 = pd.merge(merge2, card[['card_id', 'disp_id', 'card_type']], on = 'disp_id', how = 'left')
    merge4 = pd.merge(merge3, district, left_on = 'district_id_x', right_on='code')
    merge5 = pd.merge(loan, merge4, on = 'account_id')

    # Dropping some columns
    merge5.drop(['client_id', 'disp_id', 'disp_type', 'card_id', 'code'], axis = 1, inplace = True)

    # Renaming columns
    merge5.rename({
        'date_x': 'loan_date', 
        'district_id_x': 'client_district', 
        'district_id_y': 'account_district',
        'date_y': 'account_date'
    }, axis = 1, inplace = True)

    pd.set_option('display.max_columns', None)
    
    # Processing transactions information 

    trans = df_var['trans'].copy()
    trans = trans.rename({
        'type': 'trans_type',
        'date': 'trans_date', 
        'amount': 'trans_amount',
        'balance': 'trans_balance'}, axis=1)
    trans.sort_values(by = ['account_id', 'trans_date'])
    trans.drop(['trans_id', 'k_symbol', 'account', 'bank'], axis=1, inplace = True)

    #t = trans.groupby('account_id').tail(1)

    # Merging trans with previous information

    loan_info = pd.merge(merge5, trans, on = 'account_id')
    loan_info.fillna('none', inplace = True)
    #Columns that are not going to be changed
    cols = loan_info.columns.difference(['trans_date', 'trans_amount', 'trans_balance', 'trans_type', 'operation'], sort = False).tolist()

    # Group by operation

    l = loan_info.groupby(by = cols, as_index = False).agg({
                        'trans_date': ['min', 'max'],
                        'trans_amount': ['min', 'max', 'mean', 'std', 'last'],
                        'trans_balance': ['min', 'max', 'mean', 'std', 'last'],
                        'trans_type':[n_credit, n_withdrawal],
                        'operation':[n_credit_cash, n_collection_from_another_bank, 
                                    n_withdrawal_in_cash, n_remittance_to_another_bank,
                                    n_credit_card_withdrawal, n_interest_credited]
                    })

    #Change columns name 
    l.columns = ['%s%s' % (level1, '_%s' % level2 if level2 else '') for level1, level2 in l.columns]
    return l

In [ ]:
def Save_CSV(df):
    df.sort_values(by = ['loan_id'])
    df.to_csv('loan_info')

In [ ]:
def db():
    CSV_files = {'account': account, 'card': card_train, 'client': client, 'disp': disp, 'disctrict': district, 'loan': loan_train, 'trans': trans_train}
    conn = sqlite3.connect('database.db')
    # Correr só uma vez
    if not os.path.exists('./database.db'):
        for f in CSV_files:
            CSV_files[f].to_sql(f, conn)
    query = """
        SELECT loan_id, loan.account_id, status
        FROM loan, account
        WHERE loan.account_id == account.account_id
        """
         
    pd.read_sql_query(query, conn)
    
def getCorr(df, size=(11,9)):
    corr = df.corr()
    ax = plt.subplots(figsize=size)
    ax = sns.heatmap(corr,  annot = True, linewidths=.1, mask = np.triu(corr), cmap='coolwarm')

    def checkUniqueCategoricalVars():
    for f in CSV_files:
        print("{}:\n {}\n\n".format(f, CSV_files[f].select_dtypes(include=['object']).nunique()))

In [ ]:
a = pd.read_csv("results/DT_23-10.csv", delimiter=";")

In [ ]:
a.drop(a.columns.difference(['loan_id','prediction(status)']), 1, inplace=True)

In [ ]:
a = a.rename(columns={"loan_id": "Id", "prediction(status)": "Predicted"})

In [ ]:
b = a.copy()

In [ ]:
b = b.round({'Predicted': 0})

In [ ]:
b['Predicted'] = b.apply(lambda x: roundPred(x['Predicted']),axis=1)

In [ ]:
def roundPred(x):
    return 1 if (x >= 0) else -1


In [ ]:
df_all_info = pd.read_csv("results/DT_all_info.csv",delimiter=",")

In [ ]:
b.to_csv('DT_all_info_diff.csv')

In [ ]:
merge = pd.merge(b, df_all_info, on = 'Id')


In [ ]:
merge = merge.assign(diff=merge['Predicted_x'] - merge['Predicted_y'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

loan_train_1 = loan_train.copy()
for f in loan_train_1.columns:
    if loan_train_1[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(loan_train_1[f].values))
        loan_train_1[f] = lbl.transform(list(loan_train_1[f].values))

loan_test_1 = loan_test.copy()
for f in loan_test_1.columns:
    if loan_test_1[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(loan_test_1[f].values))
        loan_test_1[f] = lbl.transform(list(loan_test_1[f].values))

X = loan_test_1.drop(columns='status')
X_test = loan_train_1.drop(columns='status')
y_test = loan_train_1['status'].copy()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold

def find_best_params_kfold(model, X, y, param_grid, n_iter=10, n_splits=3):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=0)
    search = RandomizedSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, n_iter=n_iter, cv=kfold, verbose=0, random_state=0)
    result = search.fit(X_test, y_test)
    
    report(result.cv_results_)

In [ ]:
def report(results, n_top=3):
    res = []
    res_i = 0
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            res.append([np.multiply(results['mean_test_score'][candidate], -1), 
                        results['std_test_score'][candidate]])
            for key in results['params'][candidate].keys():
                res[res_i].append(results['params'][candidate][key])
            res_i+=1
    
    columns = ['Mean absolute error', 'std'] + list(results['params'][candidate].keys())
    display(pd.DataFrame(res, columns=columns))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_depth = range(2, 30, 1)
min_samples_split = range(1, 10, 1)
min_samples_leaf = range(1, 2, 1)

param_grid = dict(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
X = loan_train.loc[:, loan_train.columns != 'status']
y = loan_train.loc[:, loan_train.columns == 'status']
find_best_params_kfold(DecisionTreeClassifier(random_state=0), X_test, y_test, param_grid, 
                       n_iter=50, n_splits=10)

In [ ]:

classifier_cart = DecisionTreeClassifier(max_depth=3, min_samples_split=4, min_samples_leaf = 1, random_state=0)
classifier_cart = classifier_cart.fit(X_test, y_test)

y_test_cart = classifier_cart.predict(X)

In [ ]:
loan_train.loc[:, loan_train.columns == 'status']

In [17]:
from sklearn.preprocessing import LabelEncoder

df = loans.copy()
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))

X= df.drop(columns='status')
y = df['status'].copy()

df_test = loans_test.copy()

for f in df_test.columns:
    if df_test[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df_test[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))

X_test = df_test.drop(columns='status')
y_test = df_test['status'].copy()


NameError: name &#39;loans&#39; is not defined

In [18]:
from sklearn.model_selection import RandomizedSearchCV, KFold

def find_best_params_kfold(model, X, y, param_grid, n_iter=10, n_splits=3):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=0)
    search = RandomizedSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, n_iter=n_iter, cv=kfold, verbose=0, random_state=0)
    result = search.fit(X, y)
    
    report(result.cv_results_)

In [19]:
def report(results, n_top=3):
    res = []
    res_i = 0
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            res.append([np.multiply(results['mean_test_score'][candidate], -1), 
                        results['std_test_score'][candidate]])
            for key in results['params'][candidate].keys():
                res[res_i].append(results['params'][candidate][key])
            res_i+=1
    
    columns = ['Mean absolute error', 'std'] + list(results['params'][candidate].keys())
    display(pd.DataFrame(res, columns=columns))

In [ ]:
def cross_validationScores(modelName, model, X, y, n_folds=10):
    metrics = {'MAE': 'neg_mean_absolute_error', 
               'RMSE': 'neg_root_mean_squared_error'}
    
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    scores = cross_validate(model, X, y, cv=kfold, scoring=metrics, n_jobs=-1)

    # multiply by -1 because sklearn scoring metrics are negative
    mean_mae_score = np.multiply(scores['test_MAE'], -1).mean()
    std_mae_score = np.multiply(scores['test_MAE'], -1).std()
    mean_rmse_score = np.multiply(scores['test_RMSE'], -1).mean()
    std_rmse_score = np.multiply(scores['test_RMSE'], -1).std()

    model_score = pd.DataFrame([[mean_mae_score, mean_rmse_score]], 
                             columns=['MAE', 'RMSE'], index=[modelName])
    global models_mean_scores
    models_mean_scores = models_mean_scores.append(model_score)

    print(str(n_folds) + " fold Cross validation scores for " + modelName)
    score_df = pd.DataFrame([[mean_mae_score, std_mae_score], 
                                  [mean_rmse_score, std_rmse_score]], 
                                 columns=['Mean score', 'std'], 
                                 index=['MAE', 'RMSE'])
    display(score_df)

In [20]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, 
#	test_size = 0.30, random_state = 0)
#from sklearn import tree

#X_train = df.drop(columns='status')
#y_train = loan['status'].copy()
#clf = tree.DecisionTreeClassifier()
#clf = clf.fit(X, y)

#X_test = loan_test.drop(columns='status')
#y_test = clf.predict(X_test)

In [21]:
predict_loan = {'Id': X_test["loan_id"], 'Predicted': y_test}
df = pd.DataFrame(data=predict_loan)
df.to_csv('loan_predict.csv', index=False)

NameError: name &#39;X_test&#39; is not defined

In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_depth = range(2, 30, 1)
min_samples_split = range(1, 10, 1)

param_grid = dict(max_depth=max_depth, min_samples_split=min_samples_split)

find_best_params_kfold(DecisionTreeClassifier(random_state=0), X, y, param_grid, 
                       n_iter=50, n_splits=10)

In [22]:
classifier_cart = DecisionTreeClassifier(max_depth=2, min_samples_split=9, random_state=0)
classifier_cart = classifier_cart.fit(X, y)

y_test_cart = classifier_cart.predict(X_test)

#df_pred = pd.DataFrame(data={'Id': X_test["loan_id"], 'Predicted': y_test_cart})
#df_pred.to_csv('loan_pred_1.csv', index=False)

NameError: name &#39;DecisionTreeClassifier&#39; is not defined

In [23]:
y_test_cart = classifier_cart.predict(X_test)
#df_pred = pd.DataFrame(data={'Id': X_test_owner["loan_id"], 'Predicted': y_test_cart})
#df_pred.to_csv('loan_pred_3.csv', index=False)
y_test_cart

NameError: name &#39;classifier_cart&#39; is not defined

In [24]:
from sklearn.ensemble import BaggingClassifier
n_estimators = range(50, 1000, 50)
param_grid = dict(n_estimators=n_estimators)

find_best_params_kfold(BaggingClassifier(base_estimator=regr_cart,
                                        n_jobs=-1, random_state=0), 
                       X, y, param_grid, n_iter=30, n_splits=3)

NameError: name &#39;regr_cart&#39; is not defined

In [25]:
X_test_owner = X_test[X_test['type']==1]

NameError: name &#39;X_test&#39; is not defined

In [26]:
df_owner = df[df['type']==1]
df_owner = df_owner[['loan_id', 'loan_date', 'account_date', 'amount', 'duration', 'payments', 'account_district_id', 'client_district_id', 'status']]
X= df_owner.drop(columns='status')
y = df_owner['status'].copy()
X_test = X_test[['loan_id', 'loan_date', 'account_date', 'amount', 'duration', 'payments', 'account_district_id', 'client_district_id']]

NameError: name &#39;df&#39; is not defined

In [27]:
ax = plt.subplots(figsize=(21, 21))
ax = sns.heatmap(df.corr(), annot= True)

NameError: name &#39;df&#39; is not defined